In [1]:
!pip install sentence-transformers faiss-cpu pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 76.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [3]:
df = pd.read_excel("dataset_preguntas_respuestas_mejorado.xlsx")
df.head()


,pregunta,respuesta
0,¿Cómo puedo superar una ruptura amorosa?,"Date permiso para sentir el dolor, pero evita ..."
1,¿Qué hago si extraño a alguien que ya no está ...,Reconoce tus emociones sin juzgarte. Recordar ...
2,¿Cómo saber si alguien realmente me quiere?,"Quien te quiere te respeta, te escucha y te ac..."
3,¿Qué hago si siento que mi relación ya no avanza?,Habla con sinceridad sobre lo que te preocupa....
4,¿Cómo sanar después de una infidelidad?,"Reconoce tus emociones, establece límites y de..."


In [4]:
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Crear embeddings de todas las preguntas del dataset
embeddings = encoder.encode(df["pregunta"].tolist(), convert_to_numpy=True)
embeddings.shape


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(98, 384)

In [5]:
d = embeddings.shape[1]   # dimensión de los vectores (384)
index = faiss.IndexFlatL2(d)

index.add(embeddings)
print("Vectores indexados:", index.ntotal)


Vectores indexados: 98


In [6]:
def responder_chatbot(query, k=1):
    # Embedding de la consulta
    q_emb = encoder.encode([query], convert_to_numpy=True)

    # Buscar la pregunta más parecida
    distances, indices = index.search(q_emb, k)

    idx = indices[0][0]     # mejor match
    dist = distances[0][0]

    pregunta_encontrada = df.iloc[idx]["pregunta"]
    respuesta = df.iloc[idx]["respuesta"]

    return {
        "pregunta_encontrada": pregunta_encontrada,
        "respuesta": respuesta,
        "distancia": dist
    }


In [9]:
query = "¿Qué se necesita para ser feliz?"
respuesta = responder_chatbot(query)

print("PREGUNTA COINCIDENTE:", respuesta["pregunta_encontrada"])
print("\nRESPUESTA CHATBOT:", respuesta["respuesta"])


PREGUNTA COINCIDENTE: ¿Qué puedo hacer para mejorar mi vida en términos de fe?

RESPUESTA CHATBOT: Empieza identificando qué aspectos relacionados con fe deseas cambiar. Crea un plan concreto, habla con personas que te apoyen y toma decisiones que fortalezcan tu bienestar emocional y espiritual.
